In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ta
import pandas_datareader as pdr
import pytictoc as tt
import plotly
from tickers500 import tickers500
from tickerTA import Ticker
from tickerTA import TechnicalAnalysis
from tradingStrategy1 import TradingStrategy1

# This is to avoid the SettingWithCopyWarning in pandas
pd.options.mode.copy_on_write = True 


In [9]:
# This class is used to parse the daily data from the Yahoo Finance API
# to implement the Ticker class, TechnicalAnalysis class and TradingStrategy1 class
# After processing all the data, it will output the current Buy/Sell recommendations list
# for the 500 S&P companies
# Also it will include the price change since the last recommendation

class Recommendations:
    def __init__(self, start = pd.to_datetime('today') - pd.DateOffset(months=6), end = pd.to_datetime('today')):
        self.start = start
        self.end = end
        self.tickers500 = tickers500
        self.buy_recommendations_df = pd.DataFrame()
        self.sell_recommendations_df = pd.DataFrame()
        self.get_stock_data()
        self.current_recommendations()

    def get_stock_data(self):
        self.tradingstrategy1_df = pd.DataFrame()
        self.signals_df = pd.DataFrame()
        for ticker in self.tickers500:
            stock = Ticker(ticker, self.start, self.end)
            # stock_df = stock_df._append(stock.df)
            techA = TechnicalAnalysis(stock)
            tradingS = TradingStrategy1(techA)
            self.signals_df = self.signals_df._append(tradingS.all_signals_df)
            self.tradingstrategy1_df = self.tradingstrategy1_df._append(tradingS.trades_df)
        self.signals_df = self.signals_df.sort_values(by='Date')

    def current_recommendations(self):
        for _, row in self.signals_df.iterrows():
            if row['Buy/Sell'] == 'Buy':
                self.buy_recommendations_df = self.buy_recommendations_df._append(row)
            elif row['Buy/Sell'] == 'Sell':
                self.sell_recommendations_df = self.sell_recommendations_df._append(row)
        self.buy_recommendations_df = self.buy_recommendations_df.sort_values(by='Date', ascending=False)
        self.sell_recommendations_df = self.sell_recommendations_df.sort_values(by='Date', ascending=False)
        # self.buy_recommendations_df = self.buy_recommendations_df.drop_duplicates(subset=['Ticker'])
        # self.sell_recommendations_df = self.sell_recommendations_df.drop_duplicates(subset=['Ticker'])

In [10]:
rec = Recommendations()


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/FOX?modules=recommendationTrend&corsDomain=finance.yahoo.com&formatted=false&symbol=FOX&crumb=dhI8XUMVh5p
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/NWS?modules=recommendationTrend&corsDomain=finance.yahoo.com&formatted=false&symbol=NWS&crumb=dhI8XUMVh5p
c:\Users\chris\Documents\Unreal Projects\AlgorithmicTrading\.venv\Lib\site-packages\pandas\core\dtypes\cast.py:1057: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\chris\Documents\Unreal Projects\AlgorithmicTrading\.venv\Lib\site-packages\pandas\core\dtypes\cast.py:1081: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():


In [11]:

print(rec.signals_df)

   Ticker Buy/Sell                 Date   Price
0    VRTX      Buy  2023-11-16 00:00:00  350.72
0      CI      Buy  2023-11-16 00:00:00   282.6
0     WMT      Buy  2023-11-17 00:00:00   51.98
0    CSCO      Buy  2023-11-17 00:00:00   48.05
0     BKR      Buy  2023-11-17 00:00:00   33.68
..    ...      ...                  ...     ...
0     ACN      Buy  2024-03-22 00:00:00  350.33
2     DRI      Buy  2024-03-22 00:00:00   164.1
0    CTSH      Buy  2024-03-22 00:00:00   72.61
1     MCO     Sell  2024-03-22 00:00:00  397.52
1     STT     Sell  2024-03-22 00:00:00   76.07

[604 rows x 4 columns]


In [12]:
rec.buy_recommendations_df.to_csv('CSV/buy_recommendations.csv')
rec.buy_recommendations_df


,Ticker,Buy/Sell,Date,Price
0,CTSH,Buy,2024-03-22,72.61
2,DRI,Buy,2024-03-22,164.10
0,ACN,Buy,2024-03-22,350.33
0,ULTA,Buy,2024-03-21,523.96
0,EQIX,Buy,2024-03-21,825.00
...,...,...,...,...
0,WMT,Buy,2023-11-17,51.98
0,FANG,Buy,2023-11-17,154.03
0,CSCO,Buy,2023-11-17,48.05
0,CI,Buy,2023-11-16,282.60


In [15]:
rec.sell_recommendations_df.to_csv('CSV/sell_recommendations.csv')
rec.sell_recommendations_df


,Ticker,Buy/Sell,Date,Price
1,STT,Sell,2024-03-22,76.07
1,MCO,Sell,2024-03-22,397.52
1,TTWO,Sell,2024-03-22,152.67
1,BK,Sell,2024-03-22,57.29
1,LULU,Sell,2024-03-22,416.25
...,...,...,...,...
1,PEP,Sell,2023-12-14,171.65
1,VRSK,Sell,2023-12-14,247.81
1,MNST,Sell,2023-12-14,56.68
1,CPB,Sell,2023-12-07,43.38


In [16]:
rec.tradingstrategy1_df.to_csv('CSV/tradingstrategy1.csv')
rec.tradingstrategy1_df

,Ticker,Buy Date,Buy Price,Buy P/E Ratio,Buy Fwd P/E Ratio,Buy P/B Ratio,Buy RSI,Buy Upper BB,Buy Bollinger %b,Buy Lower BB,...,Sell Awesome Oscillator,Sell Ultimate Oscillator,Sell TSI,Sell Acum/Dist,Profit,Profit %,Duration,Ticker Cum Profit,Ticker Cum Profit %,Profitable
0,MMM,2024-01-24,96.64,0.000000,0.000000,0.000000,10.729944,115.608951,-0.312156,98.290049,...,2.664735,68.020204,5.292061,1.188619e+07,3.08,3.19,49,3.08,3.19,Yes
0,AOS,2024-02-01,78.30,24.195122,20.618938,7.141257,45.106593,82.573924,0.308838,78.197076,...,2.819618,60.015452,17.335481,5.739387e+06,6.56,8.38,33,6.56,8.38,Yes
0,AFL,2024-02-02,76.91,10.817481,12.161850,2.214446,19.802528,88.065893,-0.120995,77.891107,...,0.099588,62.241933,2.731001,3.026026e+07,5.44,7.07,34,5.44,7.07,Yes
0,ALB,2024-01-12,128.79,9.046408,12.550364,1.506945,17.171919,157.265576,-0.029841,126.554424,...,4.164324,55.076517,1.825180,-1.900730e+07,1.87,1.45,47,1.87,1.45,Yes
0,GOOGL,2024-02-29,137.28,25.950087,19.230867,6.629293,39.434912,149.522565,0.164530,136.145435,...,3.058294,55.262649,3.112388,1.675640e+08,10.88,7.93,19,10.88,7.93,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,WY,2024-01-18,32.48,30.965220,26.377777,2.538675,24.694591,35.393871,-0.005409,32.569129,...,0.540029,63.642524,10.459749,3.409682e+07,1.78,5.48,43,1.78,5.48,Yes
0,WMB,2024-01-18,33.94,14.272388,18.658537,3.764023,20.560498,36.218791,-0.043408,33.918209,...,0.875676,65.157455,11.968416,3.901996e+07,2.14,6.31,43,2.14,6.31,Yes
0,WTW,2023-12-18,238.32,27.485916,14.752159,2.942678,38.233705,248.674574,0.121565,237.217426,...,1.720706,59.741619,11.846235,3.165118e+06,9.77,4.10,25,9.77,4.10,Yes
0,YUM,2024-02-06,126.00,0.000000,0.000000,0.000000,36.524855,132.436932,0.075040,126.785068,...,2.003294,67.764283,12.374895,2.026535e+07,8.00,6.35,9,8.00,6.35,Yes
